# <strong>Intro to JupyterLab</strong>

This is a tutorial that will guide you through how to navigate through a Jupyter Notebook, as well as how to experiment with Deterlab and use basic Unix commands. While working in Deterlab, you will have access to individual computers that are called <strong>nodes</strong>. For this lab, we are working in a node called ```intro```. Future labs will contain a couple nodes, but you will only be working with one node in this notebook.

You will be working within a container called an "XDC", which stands for "eXperiment Development Container". You can have multiple XDCs, but your files are shared across XDCs. You will not need to create any XDCs besides the one that was automatically generated for you on Deterlab. The XDC that was generated for you is simply called ```xdc```. For now and in future labs, when referring to the XDC, it means your machine on Deterlab. Whenever you're located in your XDC, your command line will start with ```USERNAME_GOES_HERE@xdc```. In the left sidebar, you may move, delete, create, and even download files easily with the user interface. Currently, you cannot download/upload directories in JupyterLab.

<figure><center><img src="resources/intro/intro_xdc.png" style="width: 75%; height: 75%;"></img></center></figure>

Now, a Jupyter Notebook is an interactive Python file that you will be using for your class. A notebook has the "ipynb" extension, which stands for "interactive python notebook". Inside of the notebook contains input and output cells. You only need to be concerned about the output cells, as the input cells are pre-written for you to check your work.

Upon launching a new notebook, an input cell must be executed before you can view the output cells. <u>At the beginning of each lab</u>, click on the "fast forward" icon to run all of the cells in the notebook. View the image below to understand where to look for this button.

<figure><center><img src="resources/intro/intro_button.png" style="width: 75%; height: 75%;"></img></center></figure>

All input cells are expanded by default, and can make the notebook difficult to read. If you would like to collapse all cells, you can go to ```View > Collapse All Code``` to hide all input cells.

<figure><center><img src="resources/intro/intro_collapse.png" style="width: 50%; height: 50%;"></img></center></figure>

Upon running each step in the lab, you will notice a file called ```USERNAME_GOES_HERE_labname.tar.gz``` become created in your ```saves/``` directory. An auto-save feature was implemented in every notebook, and you will use this file as your submission for the corresponding lab. This lab can also be loaded in case you lose progress on your ```intro``` node.

The input cell that appears below this markdown cell will be ran before "Step 0". You may ignore this cell, as it's preparing libraries and functions that are required for this lab.

In [1]:
# Setting up the lab.
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
from IPython.core.magic import register_line_magic
import os
# For accessing the nodes:
import subprocess
# For the stopexp command:
import re

# These are pre-set so that the first three steps will work properly, as they all
# depend on each other. Remaining booleans will be set in the cells.
step1Complete = step2Complete = step3Complete = False

# When true, it will not auto-save at each step.
runAllSteps = False

###### Used for saving notebooks. ######
import threading
# Threading required in case steps are progressed too quickly.
save_lock = threading.Lock()

# The save function itself.
def save_notebook():
    with save_lock:
        result = subprocess.run('su - USERNAME_GOES_HERE -c "/project/USERNAME_GOES_HERE/notebooks/resources/save.py intro"', shell=True, capture_output=True, text=True)

# Creating a thread to save the notebook.
def trigger_save():
    save_thread = threading.Thread(target=save_notebook)
    save_thread.start()

###### Used for loading notebooks. ######
import queue

load_lock = threading.Lock()
result_queue = queue.Queue()

def load_notebook():
    with load_lock:
        result = subprocess.run('su - USERNAME_GOES_HERE -c "/project/USERNAME_GOES_HERE/notebooks/resources/load.py intro"', shell=True, capture_output=True, text=True)
        result_queue.put(result)  # Put the result in the queue.

# Creating a thread to load the notebook.
def trigger_load():
    load_thread = threading.Thread(target=load_notebook)
    load_thread.start()
    load_thread.join()  # Wait for the thread to complete before adding result to the queue.
    return result_queue.get()  # Get the result from the queue.

### Step 0: Starting the Lab

Click the button to begin creating the experiment.

In [2]:
# Click the button below to start the experiment.
import time
def startlab(button):
    # Defining the lab name.
    labname = "intro"

    # Writing the information to an empty field below the button.
    with output0:
        # Fixes a strange error that happens only occasionally.
        os.chdir("/project/USERNAME_GOES_HERE/notebooks")
        output0.clear_output()

        # First, checking if the materialization exists. May have been stopped by a previous lab.
        materialPattern = "real." + labname + "jup.USERNAME_GOES_HERE"

        # Listing the materializations to find if there's an existing one for this lab.
        checkMaterial = os.popen('su - USERNAME_GOES_HERE -c "mrg list materializations"').read()
        regex = re.compile(materialPattern)
        # Getting the matches:
        match = regex.search(checkMaterial)

        if match:
            display(HTML("<span style='color: orange;'>An existing activation for this lab already exists. </span><span>You might have run another \
            lab without stopping this one. Attaching the existing activation...</span>"))

            # Detaching it and re-attaching.
            subprocess.run('su - USERNAME_GOES_HERE -c "mrg xdc detach xdc.USERNAME_GOES_HERE"', shell=True, check=True)
            subprocess.run('su - USERNAME_GOES_HERE -c "mrg xdc attach xdc.USERNAME_GOES_HERE real.' + labname + 'jup.USERNAME_GOES_HERE"', capture_output=True, text=True, shell=True, check=True)
            display(HTML("<span>Re-running the installation... </span><span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

            # Re-running the installation script.
            subprocess.run('su - USERNAME_GOES_HERE -c "bash /home/runlab ' + labname + 'jup"', capture_output=True, text=True, shell=True, check=True)
            output0.clear_output()
            display(HTML("<newline><span style='color: green;'><strong>Your lab has been re-installed. </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook.</span>"))
        
        else:
            display(HTML("<span>No existing activations are found.</span>"))

            # Second, start the lab.
            display(HTML("<span>Starting the " + labname + " lab. This will take a few minutes to process. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            try:
                startexp = subprocess.run('su - USERNAME_GOES_HERE -c "bash /home/startexp ' + labname + 'jup"', capture_output=True, text=True, shell=True, check=True)
            except:
                output0.clear_output()
                display(HTML("<span style='color: red;'>There was an error starting your experiment. Make sure your password is written inside of ~/pass.txt, and try again.</span>"))
                return
            
            print(startexp.stdout, flush=True)
            output0.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(startexp.stdout, flush=True)

            # Another lab is already attached to the XDC.
            if ("XDC already attached" in startexp.stdout):
                existingLab = re.search(r"real.(.*).USERNAME_GOES_HERE", startexp.stdout).group(1)

                # Shouldn't happen.
                if labname == existingLab:
                    display(HTML("<span style='color: red;'>Your lab was already started. </span><span>Please continue to the next step.</span>"))

                # Detaching the existing lab, then attaching the current one.
                else:
                    display(HTML("<span style='color: orange;'>Warning: You did not stop your previous experiment. </span><span>Please stop your experiments \
                    before starting a new one. Detaching the " + existingLab + " experiment.</span>"))
                    subprocess.run('su - USERNAME_GOES_HERE -c "mrg xdc detach xdc.USERNAME_GOES_HERE"', shell=True, check=True)
                    display(HTML("<span>Attaching the current lab.</span>"))
                    subprocess.run('su - USERNAME_GOES_HERE -c "mrg xdc attach xdc ' + materialPattern + '"', shell=True, check=True)
    
            # Third, get the lab materials onto the node.
            display(HTML("<span>Allocating lab resources onto the node. <u>Please wait a little longer...</u></span>"))

            # Gives the notebook a couple seconds so that it will recognize the node(s).
            time.sleep(2)

            # Move the resources over.
            runlab = subprocess.run(
                'su - USERNAME_GOES_HERE -c "bash /home/runlab ' + labname + 'jup"',
                capture_output=True, text=True, shell=True
            )

            # Complete. Inform the student.
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook. Your lab will be active for one week.</span>"))


# Creating the button.
startButton = widgets.Button(description="Start Lab")

# Creating an output area.
output0 = widgets.Output()

# Run the command on click.
startButton.on_click(startlab)

# Display the output.
display(startButton, output0)

Button(description='Start Lab', style=ButtonStyle())

Output()

<hr>

If you previously stopped your lab by using the "Stop Lab" button at the bottom of the notebook, you may restore your progress below by clicking "Load Lab". <u>You do not have to load your lab if you signed out, closed your notebook, or exited your node(s) or XDC by using ```exit```.</u>

In [3]:
# Click the button below to load your lab.
def loadlab(b):
    with output0_2:
        output0_2.clear_output()
        display(HTML("<span>Searching for an existing lab in your notebook...</span>"))

    if (os.path.exists("/project/USERNAME_GOES_HERE/notebooks/saves/USERNAME_GOES_HERE_intro.tar.gz")):
        with output0_2:
            output0_2.clear_output()
            display(HTML("<span>Loading your lab...</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            result = trigger_load()
            if (result.returncode == 0):
                output0_2.clear_output()
                display(HTML("<span style='color: green;'>Your lab has been successfully loaded. Please click on the <img width='20px' height='20px' style='margin-left: 1px;' src='resources/fast_forward.png'> icon at the top of your notebook to reflect your changes.</span>"))
            elif (result.returncode == 2):
                output0_2.clear_output()
                display(HTML("<span style='color: red;'>The intro lab is inaccessible. Please start your lab. If you have already started it, wait a minute and try again.</span>"))
            else:
                output0_2.clear_output()
                display(HTML("<span style='color: red;'>An error occurred while loading your lab.</span>"))

    else:
        with output0_2:
            output0_2.clear_output()
            display(HTML("<span style='color: red;'>You do not have a saved tarball for this lab.</span>"))

# Creating the button.
loadButton = widgets.Button(description="Load Lab")

# Creating an output area.
output0_2 = widgets.Output()

# Run the command on click.
loadButton.on_click(loadlab)

# Display the output.
display(loadButton, output0_2)

Button(description='Load Lab', style=ButtonStyle())

Output()

### Step 1: Create a Folder

The ```mkdir``` command stands for "make directory". The syntax is ```mkdir name```, where ```name``` is the directory that you want to make. Note that this will make a directory inside of your current directory. You can use ```pwd``` to view where you're located, or look at the prompt in your terminal, which says: <font color="#2eeb10"><strong>USERNAME_GOES_HERE@intro</strong></font><strong>:directory</strong>, where "directory" is your current location.

If you want to create a directory in a location that's outside of your current directory, you may consider using an absolute path or use relative paths. 
- Absolute paths start with a ```/```, and you work down from the top of the directory. In the case of a home directory, the absolute path would be ```/home/USERNAME_GOES_HERE```.
- Relative paths use ```..``` to represent the parent directory of your current directory, or in other words, back one directory. An example would be ```../USERNAME_GOES_HERE```, where you go back into your ```/home``` directory, then return into your ```USERNAME_GOES_HERE``` directory. You will have more practice using path traversals in the ```pathname``` lab.

The ```mkdir``` accepts these two types of paths.

One of the most important commands to be used in Unix is the ```ls``` command. ```ls``` stands for "list", which shows all of the files and directories in your current directory. Consider using ```ls -l```, where ```-l``` stands for "long", as it displays every file and directory in your current directory, as well as permissions, links, owner, group, size, time, and name. It's not required anywhere in this lab, but you will use it everywhere on Unix.

<strong>Your task:</strong> On your XDC, sign into your ```intro``` node. To do this, type ```ssh intro```. Normally when you SSH into a computer, you would type ```ssh [user@]host```. However, since ```intro``` is recognized by your XDC as a node, you only need to type ```ssh intro```. 

You are going to be placed into ```/home/USERNAME_GOES_HERE``` (a simplified way to express your home directory is ```~```). Create a folder called ```jupyterintro``` in your home directory (```/home/USERNAME_GOES_HERE```) by using the ```mkdir``` command. Make sure that you are on your ```intro``` node first. Otherwise, you won't be able to continue.

In [4]:
# Click the button below to check your work.

# Function to check if the folder exists
def step_1():
    # Required to change boolean value.
    global step1Complete, step2Complete

    with output1:
        output1.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro /home/.checker/step1.py', shell=True)

    if (result.returncode == 1 or (result.returncode == 0 and step2Complete)):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step1Complete = True
    
    else:
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'>Check your work and try again. Did you mistype the folder name?</span>"))
            step1Complete = False

def check_step_1(b):
    step_1()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Folder")

# Creating an output area.
output1 = widgets.Output()

# Run the command on click.
button.on_click(check_step_1)

# Display the output.
display(button, output1)

Button(description='Check Folder', style=ButtonStyle())

Output()

### Step 2: Make a Text File

In your previous step, you have learned about to create a directory. To navigate into another directory, use the ```cd``` command, which stands for "change directory". The ```cd``` command accepts the same two paths that ```mkdir``` accepts: absolute and relative paths. Some examples:
- ```cd ..``` navigates into your parent directory.
- ```cd ~``` navigates into your home directory.
- ```cd /tmp``` navigates you into your ```/tmp``` directory.

A second command that you will learn about is the ```touch``` command. The ```touch``` command has the following syntax: ```touch name```, where ```name``` is the name of a file. This command will create an empty file with whatever extension you specify. Keep in mind that ```touch``` allows you to create files that do not have an extension, so it's important to specify an extension when using ```touch``` for this step.

A third command that you will learn how to use is ```chmod```. This command stands for "change mode", where the "mode" of a file is the permissions. You will learn about this significantly command in the POSIX lab, so it's not important right now to understand what it's doing. The ```chmod``` command has syntax that looks like the following: ```chmod u+x file```, where ```u``` represents "user", ```+``` represents "add the following permission", and ```x``` is the executable permission that you're adding (or removing if using ```-```). You may use this syntax for the next task.

You may read more about the [<font color="blue">touch</font>](https://www.geeksforgeeks.org/touch-command-in-linux-with-examples/) command and [<font color="blue">chmod</font>](https://www.freecodecamp.org/news/file-permissions-in-linux-chmod-command-explained/) command with these resources.

<strong>Your task:</strong> Navigate into the ```jupyterintro/``` directory that you made and create a file called ```jupytertest.txt``` by using the touch command. Try changing the permissions by using the chmod command. You can use any permissions that you'd like.

In [5]:
# Click the button below to check your work.

# Function to check if the file was created and perms were changed.
def step_2():
    # Required to change boolean value.
    global step2Complete, step3Complete

    # Loading, in case the check is slow.
    with output2:
        output2.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro /home/.checker/step2.py', shell=True)

    # Note: If already completed, it cannot be accidentally "undone" since it's part of the next step.
    if (result.returncode == 2 or step3Complete) or step2Complete:
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step2Complete = True
    elif (result.returncode == 1):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>The text file is created, but you forgot to change the permissions.</span>"))
            step2Complete = False
    else:
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>Check your work and try again.</span>"))
            step2Complete = False

def check_step_2(b):
    step_2()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check File")

# Creating an output area.
output2 = widgets.Output()

# Run the command on click.
button.on_click(check_step_2)

# Display the output.
display(button, output2)

Button(description='Check File', style=ButtonStyle())

Output()

### Step 3: Delete the Folder

Now, you are going to learn how to delete content from a Unix environment by using the ```rm``` command, which stands for "remove". The ```rm``` command works as: ```rm file```, much like the previous commands you have used for creating files and directories. The ```rm``` command works with multiple files, so to delete multiple files, you can use ```rm file1 file2 file3``` to delete multiple files at a time.

The ```rm``` command uses parameters to add more functionality to your command. Here are a few common tags:
- ```rm -i name``` indicates "interactive", where you must type "y" or "n" to indicate if the file should be deleted.
- ```rm -r name``` indicates "recursive", which can delete directories, as well as all of its contained files. Hence, the name "recursive".
- ```rm -f name``` indicates "force", which allows you to delete files that are write-protected. You can tell that a file is write-protected if you do not have write permissions on it, which you will learn about in the POSIX lab.

An additional resource for the [<font color="blue">rm</font>](https://www.howtogeek.com/858815/linux-rm-command/) command is available for you to read.

<strong>Your task:</strong> Delete the ```jupyterintro/``` directory, then check your work below.

In [6]:
# Click the button below to check your work.

# Function to check if the folder exists
def step_3():
    # Required to change boolean values.
    global step1Complete, step3Complete

    # Loading, in case the check is slow.
    with output3:
        output3.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run(['ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro /home/.checker/step3.py /home/USERNAME_GOES_HERE/jupyterintro'], shell=True)
    
    if (result.returncode == 1 and step1Complete):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>Check your work and try again.</span>"))
            step3Complete = False
    elif step1Complete == False:
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>You need to complete Step 1 before continuing.</span>"))
            step3Complete = False
    else:
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step3Complete = True

def check_step_3(b):
    step_3()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Folder")

# Creating an output area.
output3 = widgets.Output()

# Run the command on click.
button.on_click(check_step_3)

# Display the output.
display(button, output3)

Button(description='Check Folder', style=ButtonStyle())

Output()

### Step 4: Search For a File

The ```find``` command is used for locating files in a Unix environment. The syntax for the command is: ```find [path] [options] [expression]```.
- ```[path]``` is the directory that you want to search in.
- ```[options]``` allows you to set optional parameters, such as a specific name of a file (```-name pattern```), the type of file you're searching for, like regular files or directories (```-type f``` or ```-type r```), and many more.
- ```[expression]``` is the filter of the file that you wish to search for. This is where the name of the file you're searching for can be typed in, with quotes around it.

This article on the [<font color="blue">find</font>](https://www.geeksforgeeks.org/find-command-in-linux-with-examples/) command lists many different uses of the ```find``` command.

Additionally, you were reminded in Step 1 of a command called ```ls -l``` in Unix. The ```ls -al``` command is similar, but the ```-a``` means "all", which will allow you to view hidden files. Hidden files have a ```.``` in front of it, so ```.hidden_file``` is not viewable with ```ls -l```.

<strong>Your task:</strong> A file was stored in your ```intro``` node called ```.findme.txt```. Its location is unknown, so you need to use the  command to search for it. Check the top-most hierarchy of the ```intro``` node, which is ```/```. When you have found the file, delete it.

The file will be hidden, so you cannot view it with ```ls -l```. You will need to type the ```.``` in front of the file name when using the ```find``` command so that you can find it properly.

In [7]:
# Click the button below to check your work.
step4Complete = False

# Function to check if the file was deleted.
def step_4():
    # Required to change boolean values.
    global step4Complete

    # Loading, in case the check is slow.
    with output4:
        output4.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run(['ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro /home/.checker/step4'], shell=True)
    
    if (result.returncode == 1):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>Check your work and try again.</span>"))
            step4Complete = False
    elif (result.returncode == 0):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step4Complete = True

def check_step_4(b):
    step_4()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check File")

# Creating an output area.
output4 = widgets.Output()

# Run the command on click.
button.on_click(check_step_4)

# Display the output.
display(button, output4)

Button(description='Check File', style=ButtonStyle())

Output()

### Step 5: Retrieve Files from Online

Unix environments allow you to download information from online, such as website source code, RAW files, and more. To do this, we can use a command called [<font color="blue">wget</font>](https://www.geeksforgeeks.org/wget-command-in-linux-unix/). The ```wget``` command is straightforward for our needs, as it just requires a URL. The syntax is: ```wget [url]```

<strong>Your task:</strong> In your home directory, download this RAW file by using ```wget```: ```https://raw.githubusercontent.com/jesstess/Scrabble/master/scrabble/sowpods.txt```

Make sure that the file is named ```sowpods.txt```.

In [8]:
# Click the button below to check your work.
step5Complete = False

# Function to check if wget was called correctly.
def step_5():
    # Required to change boolean values.
    global step5Complete

    # Loading, in case the check is slow.
    with output5:
        output5.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run(['ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro /home/.checker/step5.py'], shell=True)

    # Triggers when wget has an issue when downloading the file for validity.
    if (result.returncode == 1):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>Error. File cannot be downloaded to check validity.</span>"))
            step5Complete = False
    # File isn't made.
    elif (result.returncode == 2):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>The sowpods.txt file cannot be found in your home directory.</span>"))
            step5Complete = False
    # In case the two files don't match.
    elif (result.returncode == 0):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>Your sowpods.txt file does not match the text file. Delete the file and try again.</span>"))
            step5Complete = False
    # Success.
    elif (result.returncode == 3):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step5Complete = True

def check_step_5(b):
    step_5()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check File")

# Creating an output area.
output5 = widgets.Output()

# Run the command on click.
button.on_click(check_step_5)

# Display the output.
display(button, output5)

Button(description='Check File', style=ButtonStyle())

Output()

### Step 6: Use a Regular Expression

Regular expressions (regex) are used to find/change strings in a file. They are in programming languages, but they are also available in Unix environments. The command, [<font color="blue">grep</font>](https://www.geeksforgeeks.org/grep-command-in-unixlinux/), supports regexes, and is used for searching and filtering text in Unix. Most programming languages support regexes, and they are supported on the Unix command-line using programs.

Some useful cases of using grep are:
- You have several .c files stored in a directory, but you want to know which .c files have the line ```#include <time.h>``` inside of them.
- You are trying to find a specific error inside of ```/var/log```.
- You want to find all occurrences of a database name in a collection of PHP files, and you want to rename them all in one command.

The ```grep``` command stands for "global regular expression print". The syntax works as follows: ```grep [options] pattern [files]```.
- ```[options]``` is a set of tags, much like the previous ```[options]``` that you have seen in the previous steps.
- ```pattern``` is the string that you wish to match. Regex is allowed in the pattern, but is not required to know for the lab.
- ```[files]``` is/are the file(s) that you wish to match with.

<strong>Your task:</strong> In the previous step, you downloaded ```sowpods.txt```. Use the ```grep``` command and find all strings that begin with the word ```CAMP``` inside of this file. Once you have printed all of the strings that start with ```CAMP```, create a text file called ```output.txt``` inside of ```/home/USERNAME_GOES_HERE```. 

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span>

- Use the following command to take your ```grep``` command and write it as a text file. This is called a redirection, which takes your output from the ```grep``` command, then writes it as a new text file: ```grep ... > /home/USERNAME_GOES_HERE/output.txt```
- You will want to find words that start with "CAMP". Regex is not required to know, but you will need to use ```^``` in front of your pattern to match with anything starting with "CAMP".
- Notice that all strings in ```sowpods.txt``` are in CAPS. You will either need to add the ```-i``` tag to your ```grep``` command (meaning case-insensitive) or set your pattern to be in CAPS.

In [9]:
# Click the button below to check your work.
step6Complete = False

# Function to check if the file was created correctly.
def step_6():
    # Required to change boolean values.
    global step5Complete, step6Complete, step7Complete

    # Loading, in case the check is slow.
    with output6:
        output6.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    result = subprocess.run(['ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro /home/.checker/step6'], shell=True)

    # If the file was moved already, change the file location. Send "1" as an argument to step6.
    if (step7Complete):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step6Complete = True
            
    # sowpods.txt doesn't exist.
    if (result.returncode == 1 or not step5Complete):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>sowpods.txt doesn't exist. Did you complete the previous step yet?</span>"))
            step6Complete = False
            
    # Check fails.
    elif (result.returncode == 0):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>Error. This check script failed. Please consult your professor/TA.</span>"))
            step6Complete = False

    # File cannot be found.
    elif (result.returncode == 2):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>Your answer cannot be found. Did you name it as output.txt?</span>"))
            step6Complete = False

    # output.txt is different from the correct answer.
    elif (result.returncode == 4):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>output.txt exists, but is incorrect.</span>"))
            step6Complete = False

    elif (result.returncode == 3):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step6Complete = True

            # For the next step, create the new folder if it doesn't exist yet.
            subprocess.run("ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro 'mkdir -p \"Important Data\"'", shell=True)
    

def check_step_6(b):
    step_6()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output6 = widgets.Output()

# Run the command on click.
button.on_click(check_step_6)

# Display the output.
display(button, output6)

Button(description='Check Work', style=ButtonStyle())

Output()

### Step 7: Moving a File

The ```mv``` command means "move". As the name implies, you can move files/directories around with this command. The syntax is: ```mv source dest```.
- ```source``` is the file that you wish to move.
- ```dest``` is the location that you wish to move the file to.

If you'd like to read more about the [<font color="blue">mv</font>](https://www.geeksforgeeks.org/mv-command-linux-examples/) command, you may follow the hyperlink.

<strong>Your task:</strong> After completing Step 6, a new folder was created in your home directory, named ```Important Data```. Move your ```output.txt``` file into this directory by using the mv command.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span>

- There is a space in your directory name. You will have to [<font color="blue">escape space characters</font>](https://linuxhandbook.com/filename-spaces-linux/) in your command.
- You can rename files use this same command! To rename a file, use ```mv oldname newname```. You are moving it to/from the same directory, but with a different name. <u>This is not required for the lab.</u>

In [10]:
# Click the button below to check your work.
step7Complete = False

# Function to check if the file was moved.
def step_7():
    # Required to change boolean value.
    global step6Complete, step7Complete

    # Loading, in case the check is slow.
    with output7:
        output7.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro "/home/.checker/step7.py \'/home/USERNAME_GOES_HERE/Important Data/output.txt\'"', shell=True)
    
    if (result.returncode == 1 and step6Complete):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step7Complete = True
    else:
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>Check your work and try again.</span>"))
            step7Complete = False

def check_step_7(b):
    step_7()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Folder")

# Creating an output area.
output7 = widgets.Output()

# Run the command on click.
button.on_click(check_step_7)

# Display the output.
display(button, output7)

Button(description='Check Folder', style=ButtonStyle())

Output()

### Step 8: Copying a File

Similar to moving files/directories, we can copy files/directories. The ```cp``` command means "copy". The syntax is the same as the ```mv``` command: ```cp source dest```.
- ```source``` is the file that you wish to copy.
- ```dest``` is the location that you wish to copy the file to.
  - ```dest``` can either end with a directory, but if you specify the path to end with a file name, it will rename the copied file to what you indicated in ```cp```.
 
You may read more about the [<font color="blue">cp</font>](https://www.geeksforgeeks.org/cp-command-linux-examples/) command here.

<strong>Your task:</strong> Now, create a copy of the file using the ```cp``` command. Call this file ```output_copy.txt```. Make sure that it remains in the same directory as your ```output.txt``` file. You should still be in your ```Important Data``` folder.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Note:</strong></span> Physically copying all the lines and creating a new text file will not work. You will need to use the copy command.

In [11]:
# Click the button below to check your work.
step8Complete = False

# Function to check if the file was copied.
def step_8():
    # Required to change boolean values.
    global step8Complete

    # Loading, in case the check is slow.
    with output8:
        output8.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run(['ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro /home/.checker/step8.py'], shell=True)

    # File wasn't copied yet.
    if (result.returncode == 0):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>output_copy.txt is not found.</span>"))
            step8Complete = False

    elif (result.returncode == 2):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>output.txt and output_copy.txt are not the same.</span>"))
            step8Complete = False

    elif (result.returncode == 3):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>Error. Please consult your professor/TA.</span>"))
            step8Complete = False
    
    elif (result.returncode == 1):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step8Complete = True

def check_step_8(b):
    step_8()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output8 = widgets.Output()

# Run the command on click.
button.on_click(check_step_8)

# Display the output.
display(button, output8)

Button(description='Check Work', style=ButtonStyle())

Output()

### Step 9: Finding the Difference Between Two Files

The ```diff``` command means "difference", which can be used to find the difference between two files. The syntax works as: ```diff [options] file1 file2```.

You will not need to use options for the ```diff``` command, but you may read more about the [<font color="blue">diff</font>](https://www.geeksforgeeks.org/diff-command-linux-examples/) command here.

<strong>Your task:</strong> Upon completing Step 8, a new folder was created inside of your ```Important Data/``` directory called ```lists/```. This contains two files: ```list1.txt``` and ```list2.txt```. One line is different between the two, very similar text files. It's too difficult to find what the difference is by hand, so we will use the diff command to find the difference between the two files.

After finding what the difference is between the two files, you will use a similar command from Step 6 to redirect the output to a text file. Call this file ```listdiff.txt```, and keep it inside of your ```lists/``` directory.

In [12]:
# Click the button below to check your work.
step9Complete = False

# Function to check the answer was correctly made.
def step_9():
    # Required to change boolean values.
    global step9Complete

    # Loading, in case the check is slow.
    with output9:
        output9.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run(['ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro /home/.checker/step9'], shell=True)

    # No attempt made.
    if (result.returncode == 3):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>listdiff.txt was not found. Is it in the correct directory?</span>"))
            step9Complete = False
            
    # Check fails.
    elif (result.returncode == 0):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>Error. This check script failed. Please consult your professor/TA.</span>"))
            step9Complete = False

    # Exists, but not correct.
    elif (result.returncode == 2):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>Your answer is incorrect. Double check your diff command.</span>"))
            step9Complete = False

    elif (result.returncode == 1):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step9Complete = True

def check_step_9(b):
    step_9()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output9 = widgets.Output()

# Run the command on click.
button.on_click(check_step_9)

# Display the output.
display(button, output9)

Button(description='Check Work', style=ButtonStyle())

Output()

### Step 10: Creating Tarballs

Now, you will create a tarball consisting of the ```Important Data/``` directory. Tarballs (```.tar.gz```) are similar to zip (```.zip```) files, except tarballs are more widely-supported in Unix environments. They are able to preserve file permissions. The ```gz``` extension means "g-zip", which is a compression type. Tarballs are usually zipped with this extension at the end of it.

Use the [<font color="blue">tar</font>](https://www.cyberciti.biz/faq/how-to-create-tar-gz-file-in-linux-using-command-line/) command to create a tarball containing the three files from above. Name the tarball ```data.tar.gz```, and make sure it's in your home directory.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> If you were to extract your tarball, the first file you should see is ```Important Data/```. It's not required that you extract your tarball, but if this step is failing to complete, this would be a good starting point to debug. If you compressed the contents inside of ```Important Data/```, rather the folder itself, then you performed the step incorrectly.

In [13]:
# Click the button below to check your work.
step10Complete = False

# Function to check if the tarball was created properly.
def step_10():
    # Required to change boolean values.
    global step10Complete

    # Loading, in case the check is slow.
    with output10:
        output10.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro /home/.checker/step10.py', shell=True, stdout=subprocess.DEVNULL)

    # File wasn't copied yet.
    if (result.returncode == 0):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>data.tar.gz was not found in your home directory.</span>"))
            step10Complete = False

    elif (result.returncode == 2):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>Your tarball was not created correctly. Review the directions and try again.</span>"))
            step10Complete = False

    elif (result.returncode == 3):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>There is an issue with how you created your tarball. Ensure that the arguments you used are -czvf for the tar command.</span>"))
            step10Complete = False

    elif (result.returncode == 1):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step10Complete = True

def check_step_10(b):
    step_10()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output10 = widgets.Output()

# Run the command on click.
button.on_click(check_step_10)

# Display the output.
display(button, output10)

Button(description='Check Work', style=ButtonStyle())

Output()

### Step 11: Secure Copying a File

We want to copy ```data.tar.gz``` to our XDC. If you forgot what your XDC is, revisit the top of your notebook. Technically, when moving files that can be copied and pasted, it is possible to copy a file by copying/pasting contents from our terminal window to a text editor. However, this isn't a viable approach when we are working with very large (and/or binary) files. We can't copy and paste a ```.tar.gz``` file from a terminal, so it will be easier to use the [<font color="blue">scp</font>](https://www.geeksforgeeks.org/scp-command-in-linux-with-examples/) command. ```scp``` stands for "secure copy", and we use this whenever we want to transfer files from one system to another.

This is the syntax: 

```scp [-args] [[user@]host1:]source_file_or_directory [[user@]host2:]destination```

This command may appear a little daunting, but it's incredibly important and useful to know. Other courses may require you to transfer files from your system to a server or vice versa. A table is provided below to describe the syntax in an easier to understand way.

|           Command             |       File I Want       |   Where To Go To     |
|:-----------------------------:|:-----------------------:|:--------------------:|
|  ```scp [-optional_args]```   |  ```~/example.txt```    |   ```intro:/tmp```   |

The final command is: ```scp ~/example.txt intro:/tmp```, which means that you want to take a file from your home directory (presumably on your XDC), then copy it to the ```intro``` node, and into the ```/tmp``` folder.

You will see that on the ```intro``` node, your terminal says <font color="#2eeb10"><strong>USERNAME_GOES_HERE@intro</strong></font>, which is a recognizable host. When you are typing commands on your XDC, <strong>USERNAME_GOES_HERE@xdc</strong> is also a recognizable host. It's good practice to look at these names on the command line so that you know what name(s) to use when using ```scp```.

Using this information, carefully type a ```scp``` command that will transfer ```/home/USERNAME_GOES_HERE/data.tar.gz``` from your ```intro``` node to ```/home/USERNAME_GOES_HERE``` on your XDC.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span>

- You will need to be on <strong>USERNAME_GOES_HERE@xdc</strong>, then using the ```scp``` command. This will mean that your source is your ```intro``` node, and your destination will be ```~```, which is the home directory on your XDC.
- When you sign in to the ```intro``` node, you are typing ```ssh intro```. The terminal recognizes that ```intro``` is a host, so you do not need to include ```user@``` when using the ```scp``` command.
- When providing a local directory to ```scp```, such as ```~```, you do not need to include the host name.

<u>The check for this step will extract your tarball and double-check that everything from Steps 5-9 were performed correctly.</u>

In [14]:
# Click the button below to check your work.
step11Complete = False

def check_tar():
    import filecmp
    
    path = os.path.expanduser("/home/USERNAME_GOES_HERE/data.tar.gz")
    imp_data = os.path.expanduser("Important Data/")
    ret = os.path.exists(path)

    if ret:
        # Create temp folder.
        temp = "/tmp/step11/"
        subprocess.run(f"mkdir -p {temp}", shell=True)

        try:
            # Check if the tarball contains all the correct contents.
            subprocess.run(f"tar -xzf {path} -C /tmp/step11", shell=True, check=True)

            # Checking if all files are in there. A more rigourous check to ensure all content is in the files.
            files = ["output.txt", "output_copy.txt", "lists/", "lists/list1.txt", "lists/list2.txt", "lists/listdiff.txt"]
            for name in files:
                name = imp_data + name
                if not os.path.exists(temp + name):
                    subprocess.run(f"rm -rf {temp}", shell=True)
                    # A file doesn't exist.
                    return 2

                if (name == "lists/list1.txt" or name == "lists/list2.txt"):
                    size = subprocess.run(f"sed -n '$=' {temp}{name}", shell=True, capture_output=True, text=True)
                    if (not size.stdout == "1000\n"):
                        return 4

                elif (name == "lists/listdiff.txt"):
                    f = open(temp + name, "r")
                    contents = f.read()
                    if (not bool(re.search(r'^\d{3}c\d{3}\n\< .{37}\n---\n> .{37}$', contents))):
                        return 5
                    f.close()

                elif (name == "output.txt" or name == "output_copy.txt"):
                    if (not filecmp.cmp(temp + "output.txt", temp + "output_copy.txt")):
                        return 6
                    f = open(temp + "output.txt", "r")
                    lines = f.readlines()
                    for line in lines:
                        if (not bool(re.search("^CAMP", line))):
                            return 7
                    f.close()
                            
            # All files exist.
            return 1

        except subprocess.CalledProcessError:
            # Not a valid .tar.gz file type.
            return 3
    else:
        # Tarball not created.
        return 0

# Function to check if the file was copied.
def step_11():
    # Required to change boolean values.
    global step11Complete

    # Loading, in case the check is slow.
    with output11:
        output11.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # Similar check to step10.py, except done locally.
    result = check_tar()

    # Clean-up.
    subprocess.run("rm -rf /tmp/step11", shell=True)

    # File wasn't copied yet.
    if (result == 0):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>data.tar.gz is not found in your home directory in your XDC.</span>"))
            step11Complete = False

    # File is missing.
    elif (result == 2):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>One or more of the files in your tarball is missing.</span>"))
            step11Complete = False

    # One of the lists is incorrect.
    elif (result == 4):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>list1.txt or list2.txt is incorrect.</span>"))
            step11Complete = False

    # listdiff.txt is incorrect.
    elif (result == 5):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>listdiff.txt is incorrect.</span>"))
            step11Complete = False

    # Files don't match.
    elif (result == 6):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>Your output.txt and output_copy.txt don't match.</span>"))
            step11Complete = False

    # Error with output.txt.
    elif (result == 7):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>Your output.txt file is incorrect.</span>"))
            step11Complete = False

    elif (result == 1):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step11Complete = True

            command = "ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro '[ -f ~/numbers.txt ] && [ \"$(wc -l < ~/numbers.txt)\" -eq 500 ] && exit 1 || exit 2'"
            answer = subprocess.run(command, shell=True)

            # If the numbers.txt file is not made yet, then run the next command.
            if (answer.returncode == 2):
                command = "ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro \"awk -v loop=500 -v range=1000 'BEGIN{srand(); do {numb = 1 + int(rand() * range); if (!(numb in prev)) {print numb; prev[numb] = 1; count++}} while (count<loop)}' > numbers.txt\""
                subprocess.run(command, shell=True)

def check_step_11(b):
    step_11()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output11 = widgets.Output()

# Run the command on click.
button.on_click(check_step_11)

# Display the output.
display(button, output11)

Button(description='Check Work', style=ButtonStyle())

Output()

### Step 12: Piping and Sorting Content

Piping refers to using the output of one command as the input to a second command. For example, using ```ls -l | grep ".txt"``` takes the output from ```ls -l```, then runs ```grep ".txt"``` on it. This returns the lines that contain ```.txt```.

You will be using the ```sort``` command for this step. The ```sort``` syntax is: ```sort [-options] input```.
- ```[-output]``` can be any set of tags, depending on what kind of output that you would like, such as reversing the sort, check if it's sorted, or specifying output to a certain file.
- ```input``` is the file that you would like to sort.

To read more about the [<font color="blue">sort</font>](https://www.geeksforgeeks.org/sort-command-linuxunix-examples/) command, you may check out this article.

<strong>Your task:</strong> <strong>Navigate back into the ```intro``` node.</strong> For this step, a text file called ```numbers.txt``` has been generated. It's located in your home directory. Using piping, run the  command to sort the text file. Put this in a new text file called ```sortednumbers.txt```. 

It is required that you use ```grep``` for this step. You will need to type the command for completing this step in the text area below. This command will be ran in the same directory as the ```numbers.txt``` file. Before submitting your answer, you're encouraged to try experimenting with ```sort``` on your ```intro``` node first. Once you feel like you have a command that works, you may input it below and see if your test passes. Recall that you are not sorting ```numbers.txt``` itself. You are sorting the text file, then outputting it to ```sortednumbers.txt```.

When you run the command in the field below, a new set of numbers will be generated and your input will be tested on it.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span>

- Upon running the ```sort``` command, you will notice that your ```sortednumbers.txt``` file is not sorted numerically. It's sorted by leading characters. <u>You don't need to worry about sorting it numerically for this exercise.</u> If you would like to sort it numerically, you can use the ```-n``` flag.
- If you need to reset the ```numbers.txt``` file, you can simply run the test without a command in the text entry.

In [15]:
# Click the button below to check your work.
step12Complete = False

# Function to check the command.
def step_12():
    # Required to change boolean values.
    global step12Complete

    # Loading, in case the check is slow.
    with output12:
        output12.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # A preliminary check before regenerating the numbers.txt file.
    if (not step11Complete):
        with output12:
            output12.clear_output()
            display(HTML("<span style='color: red;'>You must complete the previous step before attempting this.</span>"))

    elif ("\"" in userInput.value or "'" in userInput.value):
        with output12:
            output12.clear_output()
            display(HTML("<span style='color: red;'>Please do not use quotes in your answer.</span>"))

    # Otherwise, send the input to the home directory.
    else:
        # Saving their response.
        ssh_command = f'ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro "echo \'{userInput.value}\' > /home/.checker/step_12_answer.txt"'
        result = subprocess.run(ssh_command, shell=True)
    
        # Regenerate the numbers.txt file.
        command = "ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro \"awk -v loop=500 -v range=1000 'BEGIN{srand(); do {numb = 1 + int(rand() * range); if (!(numb in prev)) {print numb; prev[numb] = 1; count++}} while (count<loop)}' > numbers.txt\""
        subprocess.run(command, shell=True)
        
        if (userInput.value == "" and step11Complete):
            with output12:
                output12.clear_output()
                display(HTML("<span>numbers.txt has been reset.</span>"))

        # If the student actually has input, then we can run the test.
        else:
            # Next, make sure the command contains a pipe, does not have a comment, and contains sortedlists.txt.
            pattern = r'^(?=.*\|)(?!.*#)(?=.*sortednumbers\.txt).+$'

            if re.match(pattern, userInput.value):
                # Finally, run the command if everything looks good.
                command = 'ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro "/home/.checker/step12.py \\"' + userInput.value + '\\""'
                result = subprocess.run(command, shell=True, capture_output=True, text=True)
                
                # File wasn't copied yet. Might not happen because it gets checked in the regular expression.
                if (result.returncode == 0):
                    output12.clear_output()
                    with output12:
                        display(HTML("<span style='color: red;'>sortednumbers.txt was not found in your home directory.</span>"))
                        step12Complete = False
            
                elif (result.returncode == 2):
                    output12.clear_output()
                    with output12:
                        display(HTML("<span style='color: red;'>Your sortednumbers.txt file is incorrect.</span>"))
                        step12Complete = False

                elif (result.returncode == 3):
                    output12.clear_output()
                    with output12:
                        display(HTML("<span style='color: red;'>You're either using an unsafe command, or \"sort\" cannot be found in your input.</span>"))
                        step12Complete = False
            
                elif (result.returncode == 1):
                    output12.clear_output()
                    with output12:
                        display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
                        step12Complete = True
                        subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro "echo \'echo Congrats! You finished the lab!\' > message.sh"', shell=True)

                # This also gets reached if return code is 4.
                else:
                    output12.clear_output()
                    with output12:
                        display(HTML("<span style='color: red;'>An error occurred. Please contact your professor or TA.</span>"))
                        step12Complete = False
    
            else:
                output12.clear_output()
                with output12:
                    display(HTML("<span style='color: red;'>Check to make sure that you are using a pipe, do not have comments, and ensure that sortednumbers.txt is in your command.</span>"))
                    step12Complete = False
    
    # Auto-save.
    if (not runAllSteps):
        trigger_save()

def check_step_12(b):
    step_12()

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading12 = widgets.Output()
display(loading12)
with loading12:
    loading12.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput = widgets.Text(
    placeholder='Type a command here',
    description='Command:',
    layout=widgets.Layout(width='500px')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro "cat /home/.checker/step_12_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput.value = result.stdout[:-1]

# Creating the button.
button = widgets.Button(description="Run Command")

# After the student's response was loaded, clear the output.
loading12.clear_output()

# Creating an output area.
output12 = widgets.Output()

# Run the command on click.
button.on_click(check_step_12)

# Display the output.
display(userInput, button, output12)

Output()

Text(value='cat ~/numbers.txt | sort > sortednumbers.txt', description='Command:', layout=Layout(width='500px'…

Button(description='Run Command', style=ButtonStyle())

Output()

### Step 13: Executing Files

Not all files are executable by default. A common executable file would be a shell script, which is a file that ends with the ```.sh``` extension. When creating these files, you will have to run ```chmod``` on the files to add executable permissions to them.

In Step 2, you changed the permission of a file. A provided example from that step was ```chmod u+x file```, which was to give executable permissions only to the user. The user of a file is the person who owns the file. Using this same command, you will need to apply it to a script that was generated in your home directory. You are going to be the owner of this file, so you will not have to change the provided example too heavily to pass this step.

If you would like to learn more about the ```chmod``` command, you may follow [<font color="blue">this guide</font>](https://www.freecodecamp.org/news/linux-chmod-chown-change-file-permissions/) to learn about its usage.

<strong>Your task:</strong> Navigate to your home directory. A new file has been generated and is called ```message.sh```. You need to execute ```message.sh``` for this step. However, the permissions need to be changed because ```message.sh``` does not have the executable permission set. 

A lab called POSIX Permissions is available in a JupyterLab format, similar to this one. In case you are interested on learning more about POSIX permissions, you may read [<font color="blue">this article</font>](http://learn.leighcotnoir.com/wp-content/uploads/2016/08/POSIX-file-permissions.pdf). 

In [16]:
# Click the button below to check your work.
step13Complete = False

# Function to check the permissions.
def step_13():
    # Required to change boolean values.
    global step12Complete, step13Complete

    # Loading, in case the check is slow.
    with output13:
        output13.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@intro /home/.checker/step13.py', shell=True)

    # Step 12 wasn't done yet, so it doesn't exist.
    if (not step12Complete):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: red;'>You did not complete Step 12 yet.</span>"))
            step13Complete = False
    # File wasn't copied yet.
    if (result.returncode == 0):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: red;'>message.sh was not created correctly. Did you do the previous step?</span>"))
            step13Complete = False

    elif (result.returncode == 2):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: red;'>Your file isn't executable. Ensure that the user has executable permissions.</span>"))
            step13Complete = False

    elif (result.returncode == 1):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step13Complete = True

def check_step_13(b):
    step_13()

    # Auto-save.
    if (not runAllSteps):
        trigger_save()

# Creating the button.
button = widgets.Button(description="Check File")

# Creating an output area.
output13 = widgets.Output()

# Run the command on click.
button.on_click(check_step_13)

# Display the output.
display(button, output13)

Button(description='Check File', style=ButtonStyle())

Output()

## <strong>Grading</strong>

To check your overall grade, click on the button below.

<strong>*Warning:* Your submission will be ran on an unmodified copy of this notebook</strong>. If you tinkered with any of the input cells to pass a step, you will not get full credit.

In [17]:
# Click the button below to check your overall grade.
steps_to_check = [step_1, step_2, step_3, step_4, step_5, step_6, step_7, step_8, step_9, step_10, step_11, step_12, step_13]   

# Function to calculate grade after refreshing the cell
def calculate_grade(b):
    # To not auto-save at each step.
    global runAllSteps
    runAllSteps = True

    with gradeOutput:
        gradeOutput.clear_output()
        display(HTML("<span>Testing all steps. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # Required for checking the boolean values.
    for func in steps_to_check:
        func()  # Call each function in order

    # Assuming steps are updated above this cell in some way
    steps = [step1Complete, step2Complete, step3Complete, step4Complete, step5Complete, step6Complete, step7Complete, step8Complete, step9Complete, step10Complete, step11Complete, step12Complete, step13Complete]
    output = ""
    stepsCorrect = 0
    numOfSteps = len(steps)

    for i in range(numOfSteps):
        if steps[i]:
            stepsCorrect += 1
            output += "<div style='color: green;'>Step " + str(i + 1) + " is complete.</div>"
        else:
            output += "<div style='color: red;'>Step " + str(i + 1) + " is incomplete.</div>"

    output += "<div style='color: black;'>You have " + str(stepsCorrect) + " out of " + str(numOfSteps) + " steps completed.</div>"

    with gradeOutput:
        gradeOutput.clear_output()
        display(HTML(output))

    # Auto-save at the very end.
    runAllSteps = False
    if (not runAllSteps):
        trigger_save()

# Create a button to refresh the cell and another to calculate grade.
grade_button = widgets.Button(description="Calculate Grade")

# Link buttons to functions.
grade_button.on_click(calculate_grade)

# Output area.
gradeOutput = widgets.Output()

# Display the buttons and output.
display(grade_button, gradeOutput)

Button(description='Calculate Grade', style=ButtonStyle())

Output()

### Stopping the Lab

Once you are done with the lab, click on the "Stop Lab" button below. <strong>This will delete your materialization, which will delete all of the lab's resources.</strong> Your progress is saved automatically in ```~/notebooks/submissions```. This folder is stored on your XDC, and will not be deleted upon stopping the lab.

In [18]:
# Click the button below to stop the experiment.
def stoplab(button):
    # Check to make sure that the student wants to confirm ending the lab.
    if (confirm.value == False):
        with stop_output:
            stop_output.clear_output()
            display(HTML("<newline><span style='color: red;'>Please confirm that you wish to end the lab.</span>"))

    else:
        # Defining the lab name.
        labname = "intro"
    
        # Writing the information to an empty field below the button.
        with stop_output:
            stop_output.clear_output()
            
            display(HTML("<span>Stopping the " + labname + " lab. This will take a minute to process. Please wait.</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            stopexp = subprocess.run('su - USERNAME_GOES_HERE -c "bash /home/stopexp ' + labname + 'jup"', capture_output=True, text=True, shell=True)
            stop_output.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(stopexp.stdout)
    
            display(HTML("<newline><span style='color: green;'><strong>Your lab has been ended.</strong></span>"))

# Creating the button.
stopButton = widgets.Button(description="Stop Lab")

# Create a confirmation check.
confirm = widgets.Checkbox(
    value=False,
    description='Confirm',
    disabled=False,
    indent=False
)

# Creating an output area.
stop_output = widgets.Output()

# Run the command on click.
stopButton.on_click(stoplab)

# Display the output.
display(confirm, stopButton, stop_output)

Checkbox(value=False, description='Confirm', indent=False)

Button(description='Stop Lab', style=ButtonStyle())

Output()